##데이터 증폭 버전 이진분류 학습기 

In [ ]:
%run /content/MathUtils.ipynb
%run /content/abalone_model.ipynb

#main 함수
def binary_classification_exec(epoch_count,mb_size,report,train_ratio,val_ratio,ad_just_ratio):
  binary_load_dataset(ad_just_ratio) #증폭을 할것인지 말것인지 결정하는 ad_just_ratio
  init_param()
  train_metrics_mean_row,val_metrics_row,test_metrics = train_and_test(epoch_count,mb_size,report,train_ratio,val_ratio)
  return train_metrics_mean_row,val_metrics_row,test_metrics

#binary_load_dataset data set making 1.amplication 2.not amplication
def binary_load_dataset(ad_just_ratio):
  pulsars = []
  stars = []

  with open('/content/pulsar_stars.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)

    for row in csvreader:
      if row[8] == '1':
        pulsars.append(row)
      else:
        stars.append(row)
    
    global data,input_cnt,output_cnt
    input_cnt= 8 
    output_cnt =1  #독립변수 8개 종속변수 1개 

    star_cnt,pulsar_cnt = len(stars),len(pulsars)
'''
    for row in csvreader:
    rows.append(row) #data 한줄 한줄 저장
    
    global data, input_cnt,output_cnt

    input_cnt,output_cnt = 8,1 #독립 변수  = 8 종속변수 = 1
    data = np.asarray(rows , dtype = 'float32') #문자화 데이터 숫자화를 위한 타입 변경'''

    if ad_just_ratio: #증폭을 수행할떄 수행하면 데이터의 개수는 똑같아서 stars * 2 배 로 가능 
      data = np.zeros([star_cnt * 2,9])
      data[0:star_cnt,:] = np.asarray(stars,dtype = 'float32')

      for n in range(star_cnt):
        data[star_cnt+n] = np.asarray(pulsars[n%pulsar_cnt],dtype = 'float32') 
        #index 가 증가할때 마다 다음 데이터를 전체 유성 데이터로 나눠서 순환하는 데이터 시스템을 초기화 

      
    else:
      data = np.zeros([star_cnt+pulsar_cnt , 9]) #전체 크기에 맞는 데이터를 zeros 로 할당
      data[0:star_cnt,:] = np.asarray(stars,dtype='float32') #0~starcnt 까지는 별 데이터로 초기화 
      data[star_cnt:,:] = np.asarray(pulsars,dtype = 'float32') #starcnt ~  끝까지 는 중성자 별로 초기화 

#weight 와 bias 초기화 
def init_param():
  global weight,bias 
  weight = np.random.normal(RND_MEAN,RND_STD,size = [input_cnt,output_cnt])
  bias = np.zeros([output_cnt])

#safe_div = zero divistion 문제 해결
def safe_div(p,q):
  p,q = float(p),float(q)
  if np.abs(q)<1.0e-20: #만약 이수가 0이라면
    return np.sign(p) #p의 값을 양수 0 음수 일떄  1 0 -1 로 처리 하라 
  return p/q

#eval_accuracy_numpy 혼합 행렬로 만들어진 4가지 의 metrics score 를 return 
def eval_accuracy_numpy(output,y):
  #예측값과 실제값에 대한 초기화 참/거짓
  est_yes = np.greater(output,0)
  ans_yes = np.greater(y,0.5)

  est_no = np.logical_not(est_yes)
  ans_no = np.logical_not(ans_yes)

  #tp/tn/fp/fn 구하기
  tp = np.sum(np.logical_and(est_yes,ans_yes))
  tn = np.sum(np.logical_and(est_no,ans_no))
  fp = np.sum(np.logical_and(est_yes,ans_no))
  fn = np.sum(np.logical_and(est_no,ans_yes))

  #acc/precision/recall/f1 score 구하기 
  accuracy = safe_div(tp+tn,tp+tn+fp+fn)
  precision = safe_div(tp,tp+fp)
  recall = safe_div(tp,tp+fn)
  f1 = 2 * safe_div(recall*precision , recall+precision)

  return [accuracy,precision,recall,f1]
#arrange_data = data 미니배치 스탭 카운트 
def arrange_data(mb_size,train_ratio,val_ratio):
  global shuffle_map , val_begin_index,test_begin_index
  shuffle_map = np.arange(data.shape[0])
  np.random.shuffle(shuffle_map)

  mini_batch_step_count = int(data.shape[0] * train_ratio) //mb_size
  val_begin_index = mini_batch_step_count * mb_size
  test_begin_index = int(val_begin_index + (val_ratio * data.shape[0]))

  return mini_batch_step_count

#get train,test,val
def get_train_data(mb_size,n):

  from_idx = mb_size * n #미니 배치 사이즈의 시작
  to_idx = mb_size * (n+1) #끝

  train_data = data[shuffle_map[from_idx:to_idx]]

  return train_data[:,:-output_cnt], train_data[:,-output_cnt:]

def get_val_data():
  val_data = data[shuffle_map[val_begin_index:test_begin_index]]
  return val_data[:,:-output_cnt] , val_data[:,-output_cnt:]

def get_test_data():
  test_data = data[shuffle_map[test_begin_index:]]
  return test_data[:,:-output_cnt] , test_data[:,-output_cnt:]

#순전파 연산 
##foward_neuralnet ,forward_postproc

def forward_neuralnet(x):
  y_hat = np.matmul(x,weight)+bias 
  return y_hat,x

def forward_postproc(output,y):
  CEE = sigmoid_cross_entropy_with_logits(y,output)
  loss = np.mean(CEE)

  return loss,[y,output,CEE]


def relu(x):
  return np.maximum(x,0)

def sigmoid_cross_entropy_with_logits(z,x):
  return relu(x) - x * z + np.log(1+np.exp(-np.abs(x))) 

#역전파 연산 
## backprop_postproc , backprop_neuralnet
def backprop_neuralnet(G_output,x):
  global weight,bias #경사 하강법 진행을 위해 

  x_transpose = x.transpose()
  G_w = np.matmul(x_transpose ,G_output)
  G_b = np.sum(G_output,axis = 0)

  weight -= LEARNING_RATE * G_w
  bias -= LEARNING_RATE * G_b


def backprop_postproc(aux_pp_y_output_CEE):
  y,output,CEE = aux_pp_y_output_CEE

  g_loss_entropy = 1.0 / np.prod(CEE.shape)
  g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y,output)

  G_output = g_loss_entropy * g_entropy_output
  return G_output

def sigmoid(x):
  return np.exp(-relu(x))/(1.0+np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z,x):
  return - z + sigmoid(x)

def run_train(x,y):
  y_hat,aux_nn_x = forward_neuralnet(x)
  loss,aux_pp_y_output_CEE = forward_postproc(y_hat,y)
  acc = eval_accuracy_numpy(y_hat,y)

  G_output = backprop_postproc(aux_pp_y_output_CEE)
  backprop_neuralnet(G_output,aux_nn_x)
  return loss,acc

def run_test(x,y):
  y_hat,_ = forward_neuralnet(x)
  loss,_ = forward_postproc(y_hat,y)
  acc = eval_accuracy_numpy(y_hat,y)
  return loss,acc

def train_and_test(epoch_count,mb_size,report,train_ratio,val_ratio):
  mini_batch_step_count = arrange_data(mb_size,train_ratio,val_ratio)
  test_x,test_y = get_test_data()
  val_x,val_y = get_val_data()

  losses_mean_row = [] #loss 값 평균 저장 
  val_loss_row = []

  for epoch in range(epoch_count):
    losses = [] #loss 값 저장 

    for n in range(mini_batch_step_count):
      train_x,train_y = get_train_data(mb_size,n)

      loss,_ = run_train(train_x,train_y)
      losses.append(loss)

    val_loss,val_acc = run_test(val_x,val_y)
    val_loss_row.append(val_loss)

    if report>0 and (epoch+1) % report == 0:
            print("Epoch {} : Train - Loss = {:.3f} / Val - Loss = {:.3f}, Acc = {:.3f}, Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3F}".\
                  format(epoch+1, np.mean(losses), val_loss, val_acc[0], val_acc[1], val_acc[2], val_acc[3]))
    
    losses_mean = np.mean(losses)
    losses_mean_row.append(losses_mean)

  test_loss,test_acc = run_test(test_x,test_y)

  print("\n","=" * 50, 'Final Test', '=' * 50)
  print('\nTest Acc = {:.3f}, Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3F}'.\
        format(test_acc[0], test_acc[1], test_acc[2], test_acc[3]))
  print('\nLoss = {:.3f}'.format(test_loss))

  return [losses_mean_row],[val_loss_row],[test_loss,test_acc]

In [ ]:
binary_load_dataset(ad_just_ratio=True)

In [ ]:
train_x,train_y = get_train_data(6,0)
val_x,val_y = get_val_data()
test_x,test_y = get_test_data()

print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)
print(test_x.shape)
print(test_y.shape)

(6, 8)
(6, 1)
(6503, 8)
(6503, 1)
(6509, 8)
(6509, 1)


In [ ]:
loss,acc = run_train(train_x,train_y)
val_loss,val_acc = run_test(val_x,val_y)
test_loss,test_acc = run_test(test_x,test_y)

print(loss,acc)
print(val_loss,val_acc)
print(test_loss,test_acc)

532.1588842329131 [0.3333333333333333, 0.0, 0.0, 0.0]
504.2782198159135 [0.5069967707212056, 0.0, 0.0, 0.0]
508.48401400331153 [0.5039176524811799, 0.0, 0.0, 0.0]


In [ ]:
mini_batch_step_count = arrange_data(6,0.6,0.2)

In [ ]:
init_param()

In [ ]:
train_metrics_mean_row, val_metrics_row, test_metrics = binary_classification_exec(epoch_count = 100, 
                                                                                   mb_size     = 32, 
                                                                                   report      = 1, 
                                                                                   train_ratio = 0.6, 
                                                                                   val_ratio   = 0.2,
                                                                                   ad_just_ratio = True)

Epoch 1 : Train - Loss = 64877.655 / Val - Loss = 128191.972, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 2 : Train - Loss = 195491.199 / Val - Loss = 256430.193, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 3 : Train - Loss = 326104.833 / Val - Loss = 384668.414, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 4 : Train - Loss = 456718.466 / Val - Loss = 512906.635, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 5 : Train - Loss = 587332.100 / Val - Loss = 641144.856, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 6 : Train - Loss = 717945.733 / Val - Loss = 769383.077, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 7 : Train - Loss = 848559.367 / Val - Loss = 897621.298, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 8 : Train - Loss = 979173.000 / Val - Loss = 1025859.518, Acc = 0.505, Precision = 0.505, Recall = 1.000, F1 = 0.671
Epoch 9 : Train - Loss =

In [5]:
age = [10,30,20,40,90,70,60]
gender =['M',"F",'M','F','M','F','M']

import pandas as pd 



ValueError: ignored